# Generate QnA synthetic dataset from CSV


In [1]:
from dotenv import load_dotenv

import os, shutil, random
from langchain_community.document_loaders.csv_loader import CSVLoader
from util.preprocess import convert_html_to_md

load_dotenv()
raw_data_dir = "raw_data"
file_path = f"{raw_data_dir}/prod-st-excel/pivot_spec_mx_smartphone_s24.csv"


## 1. Read & Preprocess CSV file
---

In [2]:
%%time

loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },    
)
docs = loader.load()

CPU times: user 7.09 ms, sys: 4.58 ms, total: 11.7 ms
Wall time: 24.4 ms


In [3]:
preprocessed_docs = []
for doc in docs:
    md = convert_html_to_md(doc.page_content)
    preprocessed_docs.append(md)

print(preprocessed_docs[0])

모델코드: SM-S921NLBEKOO
운영체제: Android
센서: 가속도 센서,기압 센서,지문 센서,자이로 센서,지자기 센서,홀 센서,조도 센서,근접 센서
S펜 지원: 없음
디스플레이\_크기 (Main Display): 156.4 mm
디스플레이\_해상도 (Main): 2340 x 1080 (FHD+)
디스플레이\_종류 (Main): Dynamic AMOLED 2X
디스플레이\_색심도 (Main): 16 M
프로세서\_CPU 속도: 3.2 GHz,2.9 GHz,2.6 GHz,1.95 GHz
프로세서\_CPU 종류: Deca-Core
카메라\_후면 카메라 - 화소 (Multiple): 50.0 MP + 10.0 MP + 12.0 MP
카메라\_후면 카메라 - 조리개 값 (Multiple): F1.8 , F2.4 , F2.2
카메라\_후면 카메라 - 오토 포커스: 예
카메라\_후면 카메라 - OIS: 예
카메라\_후면 카메라 줌: 없음
카메라\_전면 카메라 - 화소: 12.0 MP
카메라\_전면 카메라 - 조리개 값: F2.2
카메라\_전면 카메라 - 오토 포커스: 예
카메라\_전면 카메라 - OIS: 없음
카메라\_후면 카메라 - 플래쉬: 예
카메라\_전면 카메라 - 플래쉬: 없음
카메라\_동영상 녹화 해상도: UHD 8K (7680 x 4320) @30fps
메모리/스토리지(저장 용량)\_메모리 (GB): 8 GB
메모리/스토리지(저장 용량)\_스토리지(저장 용량) (GB): 256 GB
메모리/스토리지(저장 용량)\_사용 가능한 스토리지(저장 용량) (GB): 232.7 GB
메모리/스토리지(저장 용량)\_외장 스토리지(저장 용량) 지원: 없음
네트워크\_SIM 개수: Dual-SIM
네트워크\_SIM 사이즈: 없음
네트워크\_SIM 슬롯 타입: SIM 1 + eSIM / Dual eSIM
네트워크 (S/W 사용)\_2G GSM: GSM850,GSM900,DCS1800,PCS1900
네트워크 (S/W 사용)\_3G UMTS: B1(2100),B2(1900

## 2. Construct QnA Pairs
---

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair

llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=1024,
    openai_api_version="2024-05-01-preview",
    azure_deployment="gpt-4o"                       
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template()
chain = prompt | llm | parser

Prepare input batch

In [5]:
input_batch = []

for doc in preprocessed_docs:
    dic = {"context": doc, "domain": "Samsung Galaxy S series Smartphone", "num_questions": "1"}
    input_batch.append(dic)

In [6]:
%%time
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})

CPU times: user 769 ms, sys: 33 ms, total: 802 ms
Wall time: 2min 21s


In [7]:
# Remove duplicate items
qa_pair = [dict(t) for t in {tuple(d.items()) for d in qa_pair}]


## 3. Save to jsonl for fine-tuning
---

In [9]:
qa_pair

[{'QUESTION': '갤럭시 S24 통신사폰(SM-S921NZAFKOD)의 후면 카메라의 주요 사양은 무엇인가요?',
  'ANSWER': '갤럭시 S24 통신사폰(SM-S921NZAFKOD)의 후면 카메라는 50.0 MP, 10.0 MP, 12.0 MP의 화소를 가지며, 조리개 값은 F1.8, F2.4, F2.2입니다. 또한, 오토 포커스와 OIS를 지원하며, 3배 광학 줌과 최대 30배 디지털 줌을 제공합니다.'},
 {'QUESTION': '갤럭시 S24+ (SM-S926NZKAKOO)의 후면 카메라의 주요 특징은 무엇인가요?',
  'ANSWER': '갤럭시 S24+ (SM-S926NZKAKOO)의 후면 카메라는 50.0 MP, 10.0 MP, 12.0 MP의 화소를 가지며, 조리개 값은 각각 F1.8, F2.4, F2.2입니다. 또한, 오토 포커스와 OIS를 지원하며, 3배 광학 줌과 최대 30배 디지털 줌을 제공합니다.'},
 {'QUESTION': '갤럭시 S24 Ultra의 후면 카메라의 화소와 조리개 값은 어떻게 되나요?',
  'ANSWER': '갤럭시 S24 Ultra의 후면 카메라는 200.0 MP (F1.7), 50.0 MP (F3.4), 12.0 MP (F2.2), 10.0 MP (F2.4)로 구성되어 있습니다.'},
 {'QUESTION': '갤럭시 S24 자급제 모델(SM-S921NZVFKOO)의 후면 카메라의 주요 사양은 무엇인가요?',
  'ANSWER': '갤럭시 S24 자급제 모델의 후면 카메라는 50.0 MP, 10.0 MP, 12.0 MP의 화소를 가지고 있으며, 조리개 값은 F1.8, F2.4, F2.2입니다. 또한 오토 포커스와 OIS를 지원하며, 3배 광학 줌과 최대 30배 디지털 줌을 제공합니다.'},
 {'QUESTION': '갤럭시 S24 자급제 모델(SM-S921NZOEKOO)의 후면 카메라의 주요 사양은 무엇인가요?',
  'ANSWER': '갤럭시 S24 자급제 모델의 후면 카메라는 50.0 MP, 

In [11]:
def convert_to_oai_format(qa_pair, system_prompt_msg="You're an AI assistant that guides a user to the location of your CS center"):
    """
    Convert the QA pair to the jsonl format required by the OpenAI API.

    Args:
        qa_pair: list of dictionaries or list of lists containing the QA pairs
        system_prompt_msg: message to be displayed as the system prompt

    Returns:
        formatted_data: jsonl format data for OpenAI API
    """
    if isinstance(qa_pair, list):
        formatted_data = []
        for qa in qa_pair:

            sample = [{"role": "system", "content": system_prompt_msg}]

            if isinstance(qa, list): # multi-turn
                for qa_ in qa:
                    if isinstance(qa_, dict):
                        user_message = {"role": "user", "content": qa_["QUESTION"]}
                        assistant_message = {"role": "assistant", "content": qa_["ANSWER"]}
                    else:
                        user_message = {"role": "user", "content": qa_[0]}
                        assistant_message = {"role": "assistant", "content": qa_[1]}
                    sample.append(user_message)
                    sample.append(assistant_message)
            else:  # single turn
                if isinstance(qa, dict):                
                    user_message = {"role": "user", "content": qa["QUESTION"]}
                    assistant_message = {"role": "assistant", "content": qa["ANSWER"]}
                else:
                    user_message = {"role": "user", "content": qa[0]}
                    assistant_message = {"role": "assistant", "content": qa[1]} 
                sample.append(user_message)
                sample.append(assistant_message)

            msg = {"messages": sample} 
            formatted_data.append(msg)
        random.shuffle(formatted_data) 
        return formatted_data

    else:
        print("Argument is not a list")
        return None
    

In [12]:
import json
#from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = "You are an AI assistant that guides product specifications for the Galaxy series."
#system_prompt_msg = "You're an AI assistant that guides a user to general information of Galaxy Series."
save_filename = "cs-prod-spec-s24"
oai_qa_pair = convert_to_oai_format(qa_pair)

#save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")